# Génération de données ecommerces

In [ ]:
pip install faker kafka-python

In [ ]:
from kafka import KafkaProducer
from faker import Faker
import json, random, time

fake = Faker()
BROKER = "broker:29092"         # Adresse interne Docker
TOPIC = "ecommerce_actions"     # Nom du topic

ACTIONS = ["VIEW_PRODUCT", "ADD_TO_CART", "PURCHASE"]

producer = KafkaProducer(
    bootstrap_servers=[BROKER],
    value_serializer=lambda v: json.dumps(v).encode("utf-8")
)

print(f"Envoi d'événements aléatoires sur le topic '{TOPIC}'...")

while True:
    event = {
        "event_id": fake.uuid4(),
        "timestamp": fake.iso8601(),
        "user_id": f"user_{random.randint(1, 1000)}",
        "session_id": fake.uuid4(),
        "action_type": random.choices(ACTIONS, weights=[0.6, 0.3, 0.1])[0],
        "product_id": f"PROD-{random.randint(100, 999)}",
        "product_name": fake.word().capitalize(),
        "price": round(random.uniform(10, 300), 2),
        "ip_address": fake.ipv4_public(),
        "user_agent": fake.user_agent()
    }

    producer.send(TOPIC, event)
    print("Événement envoyé :", event)
    time.sleep(0.2)  # 5 événements/seconde

# Interruption: Ctrl+C pour arrêter
